# Exam Bonus Problem - Decision Tree Learning

Professor: Prof. Dr. Patrick  Glauner<br>
Participant: 22211158 -> Noah Tjorven Burdorf

#### Remark

This submission is additional to another one in which I participated.<br>
My main goal is to get potential feedback to my algorithm which differs severely from the collaborated one.

Regarding the intent of the submission the overhead is minimized and the visualisation is drastically reduced.

## Overhead

There are a few things to be mentioned regarding the dataset and visualisation.
This project is divided in a few files, but for simplicity we view it all as a simple script.
Used libraries for this solution are pprint, math and random.

In [1]:
from pprint import pprint
import math
import random

### The Dataset
The dataset is a simple list containing a dictionary with the keys "result" and "dependencies".<br>
Here the dependencies are representing a table, a method for reading a csv for data acquisition can be found in the other submission.
The data are from the lecture where we initially got to know the DT_LEARNING function.

In [2]:
examples = [
    {"dependencies": {"visible": 0, "distance": "< 10", "armed": 0, }, "result": True},
    {"dependencies": {"visible": 0, "distance": "< 10", "armed": 1, }, "result": False},
    {"dependencies": {"visible": 0, "distance": "10 - 20", "armed": 0}, "result": False},
    {"dependencies": {"visible": 0, "distance": "10 - 20", "armed": 1}, "result": False},
    {"dependencies": {"visible": 0, "distance": "> 20", "armed": 0, }, "result": False},
    {"dependencies": {"visible": 0, "distance": "> 20", "armed": 1, }, "result": False},
    {"dependencies": {"visible": 1, "distance": "< 10", "armed": 0, }, "result": True},
    {"dependencies": {"visible": 1, "distance": "< 10", "armed": 1, }, "result": True},
    {"dependencies": {"visible": 1, "distance": "10 - 20", "armed": 0}, "result": True},
    {"dependencies": {"visible": 1, "distance": "10 - 20", "armed": 1}, "result": True},
    {"dependencies": {"visible": 1, "distance": "> 20", "armed": 0, }, "result": False},
    {"dependencies": {"visible": 1, "distance": "> 20", "armed": 1, }, "result": False}
]

The attributes are additionally stored in another list of dictionaries, consisting of the keys "name" and "values".

In [3]:
attributes = [
    {"name": "visible", "values": [0, 1]},
    {"name": "distance", "values": ["< 10", "10 - 20", "> 20"]},
    {"name": "armed", "values": [0, 1]}
]

### The Visualisation
The function to plot, or more exactly print the decision tree is a provisional. An advanced tree is part of the other submission.
An additional overview over the received data is provided by the pretty print library. 

In [4]:
def print_tree(data: dict | bool, depth: int, decision):
    if type(data) is bool:
        print((depth - 1) * "    " + "|-- " + str(decision) + bool(depth) * " -> " + str(data))
        return
    print((depth - 1) * "    " + '|-- ' + str(decision) + bool(depth) * " -- " + data["attribute"])
    data.pop("attribute")
    for key, value in data.items():
        print_tree(value, depth + 1, key)

## The Algorithm

### Implementation of the predefined methods
First of all we take a look at the implementation of the three methods "DT_LEANING", "PLURALITY_VAL" and "IMPORTANCE".<br>
A few functionalities are outsourced, but all methods are named intuitively, and are going to be explained in the course of this documentation.

In [5]:
def DT_LEARNING(examples: list[dict], attributes: list[dict], parent_examples: list[dict]) -> bool | dict:
    """recursive decision tree learning algorithm as discussed in the lecture"""
    tree = {}

    if not examples:
        return PLURALITY_VAL(parent_examples)
    elif _same_classification(examples):
        return examples[0]["result"]
    elif not attributes:
        return PLURALITY_VAL(examples)
    else:
        attribute = _argmax_importance(attributes, examples)
        tree["attribute"] = attribute["name"]
        attributes.remove(attribute)
    for value in attribute["values"]:
        child_examples = _determine_child_examples(attribute["name"], value, examples)

        tree[value] = DT_LEARNING(child_examples, attributes.copy(), examples)

    return tree

This function is as explained in the lecture, extended with formalized python syntax.

In [6]:
def PLURALITY_VAL(examples: list[dict]) -> bool:
    """calculates the most common result in the examples"""
    true_examples = _count_true_examples(examples)
    return true_examples > len(examples) / 2

Implementation of the plurality functionality. As said the used function will be explained.

In [7]:
def IMPORTANCE(attribute: dict, examples: list[dict]) -> float:
    """computes the importance of an attribute"""
    D = _count_true_examples(examples) / len(examples)
    information_gain = _binary_entropy(D)

    for value in attribute["values"]:
        occurrence, positive_occurrence = _attribute_occurrence(examples, attribute["name"], value)
        remainder = occurrence / len(examples) * _binary_entropy(positive_occurrence / occurrence)
        information_gain -= remainder

    return information_gain

The information-gain is calculated with the least amount of lines of code, without compromising the readability.

### Helper methods

First there is the "_count_true_examples" method, which like the name suggests, counts all examples with a positive result.

In [8]:
def _count_true_examples(examples: list[dict]) -> int:
    """counts the examples with the output value true"""
    true_count = 0
    for example in examples:
        if example["result"]:
            true_count += 1

    return true_count

Secondly we consider the "_binary_entropy" which is calculated following the formula, except for the input-values zero and one, where the value zero is returned, avoiding an error.

In [9]:
def _binary_entropy(q: float) -> float:
    """calculates the binary entropy of input q"""
    return 0 if q == 1 or q == 0 else -(q * math.log2(q) + (1 - q) * math.log2(1 - q))

After that we have the function "_attribute_occurrence", returning a tuple consisting of the total occurrence of an attribute and the corresponding occurrences with a positive result in the examples.
This was combined to prevent repetition and a higher runtime.

In [10]:
def _attribute_occurrence(examples: list[dict], attribute_name: str, attribute_value: any) -> tuple[int, int]:
    """returns the overall and positive occurrence of an attribute in the given examples"""
    occurrence = 0
    positive_occurrence = 0

    for example in examples:
        if example["dependencies"][attribute_name] == attribute_value:

            occurrence += 1
            if example["result"]:
                positive_occurrence += 1

    return occurrence, positive_occurrence

Now we had to implement "_same_classification" which checks if all examples have the same result.

In [11]:
def _same_classification(examples: list[dict]) -> bool:
    """returns if all examples lead to the same result"""
    classification = examples[0]["result"]
    for example in examples:
        if example["result"] != classification:
            return False
    return True

In order to recurse the "DT_LEARNING" we have to reduce the examples according to our new attribute, which is done as follows.

In [12]:
def _determine_child_examples(attribute_name: dict, attribute_value: str, examples: list[dict]) -> list[dict]:
    """filters examples for an specific attribute value"""
    child_examples = []

    for example in examples:
        if example["dependencies"][attribute_name] == attribute_value:
            child_examples.append(example)

    return child_examples

The last unexplored step of this algorithm is the argmax function of the IMPORTANCE, named according to its functionality "_argmax_importance".<br>
It loops over all attributes and calculates the number and name of the attributes affecting the result the most. In case of a tie it breaks it randomly, which is the only use for the random function in this program.

In [13]:
def _argmax_importance(attributes: list[dict], examples: list[dict]) -> dict:
    """equivalent of an argmax function for the IMPORTANCE function"""
    maximum_importance = 0
    maximum_attributes = []

    for attribute in attributes:
        importance = IMPORTANCE(attribute, examples)

        if importance > maximum_importance:
            maximum_importance = importance
            maximum_attributes = [attribute]

        elif importance == maximum_importance:
            maximum_attributes.append(attribute)

    return random.choice(maximum_attributes)

## The Result

With the output of the DT_LEANING algorithm, we can create a simple tree.
In this case it is not pretty but does the job. 

In [14]:
result = DT_LEARNING(examples, attributes, examples)

print_tree(result.copy(), 0, "")
print("\n")
pprint(result)

|-- distance
|-- < 10 -- visible
    |-- 0 -- armed
        |-- 0 -> True
        |-- 1 -> False
    |-- 1 -> True
|-- 10 - 20 -- visible
    |-- 0 -> False
    |-- 1 -> True
|-- > 20 -> False


{'10 - 20': {0: False, 1: True},
 '< 10': {0: {0: True, 1: False}, 1: True},
 '> 20': False,
 'attribute': 'distance'}


First there is my own visualisation, with the attribute named on top, and the values called out on each branch. Followed by the child decision tree and eventually the classification, labeled with an additional "->".<br>
Because of the awful visualisation, I added a pretty-printed version of the raw output right behind.

## Sources

Ideas can be influenced by the other approach.<br>
There were no external sources utilized beside the lecture material.<br>
I emphasize that no AI was used, and all the code - except the DT_LEARNING - is entirely self written.